# Script to extract and aggregate call stack infomation for dumps in given folder.

## Set Variables

In [9]:
using System.Diagnostics;
using System.IO;
using System.Text.Json;
using System.Text.RegularExpressions;


string windbgPath = "C:\\Users\\v-bibu\\windbg\\windbg.exe";
string dumpFolder = @"Q:\dumps-testhang";
List<string> debugCommandList = new List<string>(){
    ".symfix",
    "k",
    "q"
};

## Analyze class

In [10]:

public class DumpAnalyzer
{
    #nullable enable
    private string? windbgPath;

    public DumpAnalyzer(string windbgPath) 
    {
        if (String.IsNullOrEmpty(windbgPath))
            throw new ArgumentException($"Path to windbg.exe should be provided.");

        if (!Path.Exists(windbgPath))
            throw new ArgumentException($"The given windbg path {windbgPath} doesn't exist.");

        this.windbgPath = windbgPath;
    }

    public void DebugDumpWithWinDBG(List<string> debugCommandList, string dumpPath, string outputPath)
    {
        var debugCommandSequence = debugCommandList
            .Select(command => $"{command}\n")
            .Aggregate(String.Concat);

        try
        {
            Console.WriteLine($"Start to debug dump: {dumpPath}.");
            using (Process process = new Process())
            {
                process.StartInfo.FileName = windbgPath;
                process.StartInfo.Arguments = $"-z {dumpPath} -c \"{debugCommandSequence}\" -logo {outputPath}";
                process.StartInfo.UseShellExecute = false;
                process.Start();

                process.WaitForExit();
            }
            Console.WriteLine($"Write debug output to: {outputPath}.");
        }
        catch (Exception ex)
        {
            Console.WriteLine(ex.Message);
        }
    }

    public string? ExtractCallStackFromDebugOutput(string debugOutput)
    {
        string pattern = @"# Child-SP\s+RetAddr\s+Call Site\s+(.*?)\d+:\d+>\s.?";
        Match match = Regex.Match(debugOutput, pattern, RegexOptions.Singleline);
        if (match.Success)
        {
            return match.Groups[1].Value.Trim();
        }
        else
        {
            Console.WriteLine($"can't extract call stack info from debug output.");
            return null;
        }
    }

    public void SaveCallStack(List<string> debugCommandList, string dumpFolder)
    {
        foreach (string dumpPath in Directory.GetFiles(dumpFolder, "stress*.dmp"))
        {
            Console.WriteLine($"====== Debugging {dumpPath} and extracting call stack ======");
            try
            {
                string outputPath = dumpPath.Replace(".dmp", ".txt");
                DebugDumpWithWinDBG(debugCommandList, dumpPath, outputPath);

                string debugOutput = File.ReadAllText(outputPath);
                string? callStackInfo = ExtractCallStackFromDebugOutput(debugOutput);
                if (string.IsNullOrEmpty(callStackInfo))
                    continue;

                File.WriteAllText(outputPath, callStackInfo);
            }
            catch (Exception ex)
            {
                Console.WriteLine($"fail to save call stack for {dumpPath}: {ex.Message}");
            }
        }
    }

    public (string?, int?) GetFileNameAndLineNumberOfTopStackFrame(string keyword, string callStack)
    {
        string[] lines = callStack.Split(new[] { '\r', '\n' });
        string lineContainKeyword = lines.FirstOrDefault(line => line.Contains(keyword), String.Empty);

        if (String.IsNullOrEmpty(lineContainKeyword))     
        {
            Console.WriteLine($"Fail to find the keyword in call stack.");
            return (null, null);
        }

        // look for the pattern [filename@linenumber]
        string pattern = @"\[(.*?)\]";
        Match match = Regex.Match(lineContainKeyword, pattern, RegexOptions.Singleline);

        if (!match.Success)
        {
            Console.WriteLine($"The symbol is not available.");
            return (null, null);
        }
        
        string fileNameWithLineNumber = match.Groups[1].Value.Trim();
        string[] splitOutput = fileNameWithLineNumber.Split("@");

        string? fileName = splitOutput.FirstOrDefault(String.Empty);
        if (String.IsNullOrEmpty(fileName))
        {
            Console.WriteLine($"Fail to extract filename.");
            fileName = null;
        }

        string? lineNumberstr = splitOutput.LastOrDefault(String.Empty).Trim();
        if (String.IsNullOrEmpty(lineNumberstr))
        {
            Console.WriteLine($"Fail to extract line number.");
            return (fileName, null);
        }

        bool success = int.TryParse(lineNumberstr, out int lineNumber);
        if (!success)
        {
            Console.WriteLine($"Fail to parse line number.");
            return (fileName, null);
        }

        return (fileName, lineNumber);

    }

    public void GetAndSaveKey(string keyword, string debugOutputFolder)
    {
        string[] callStackFileList = Directory.GetFiles(debugOutputFolder, "stress*.txt");
        Dictionary<string, List<string>> keyToPath = new();


        foreach (string callStackFile in callStackFileList)
        {
            Console.WriteLine($"====== Processing {callStackFile} ======");

            try
            {
                string callStackInfo = File.ReadAllText(callStackFile);

                (string? fileName, int? lineNumber) = GetFileNameAndLineNumberOfTopStackFrame(keyword, callStackInfo);

                if (fileName == null || lineNumber == null)    
                {
                    if (!keyToPath.TryGetValue("unknown", out var unknownKVP))
                    {
                        keyToPath["unknown"] = new List<string>();
                    }
                    keyToPath["unknown"].Add(callStackFile);
                    continue;
                }

                string srcLine = File.ReadAllLines(fileName)[lineNumber.Value - 1];

                if (!keyToPath.TryGetValue(srcLine, out var kvp))
                {
                    keyToPath[srcLine] = new List<string>();
                }

                keyToPath[srcLine].Add(callStackFile);
            }

            catch (Exception ex)
            {
                Console.WriteLine(ex.Message);
            }
        }

        string jsonString = JsonSerializer.Serialize(keyToPath, new JsonSerializerOptions
        {
            WriteIndented = true // Makes the JSON output more readable with indentation
        });

        // Write JSON string to a file
        string filePath = Path.Combine(debugOutputFolder, "dictionary.json");
        File.WriteAllText(filePath, jsonString);
    }
}



## Run

In [11]:
var analyzer = new DumpAnalyzer(windbgPath);
analyzer.SaveCallStack(debugCommandList, dumpFolder);
analyzer.GetAndSaveKey("::gc_heap::", dumpFolder);

====== Debugging Q:\dumps-testhang\stress_1e68_2024-07-16_01-51-14-392_44bc.dmp and extracting call stack ======
Start to debug dump: Q:\dumps-testhang\stress_1e68_2024-07-16_01-51-14-392_44bc.dmp.
Write debug output to: Q:\dumps-testhang\stress_1e68_2024-07-16_01-51-14-392_44bc.txt.
====== Processing Q:\dumps-testhang\stress_1e68_2024-07-16_01-51-14-392_44bc.txt ======
Fail to find the keyword in call stack.
